# Example queries for biological entities on COVID-19 Knowledge Graph
[Work in progress]

This notebook demonstrates how to run Cypher queries in a Jupyter Notebook by connecting to a database server.

In [1]:
import datetime
import pandas as pd
from py2neo import Graph

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

#### Connect to COVID-19-Net Knowledge Graph

In [3]:
graph = Graph("bolt://132.249.238.185:7687", user="reader", password="demo")

### List Node Metadata

In [4]:
query = """
MATCH (n:NodeMetadata)
RETURN n.name, n.shortDescription, n.description, n.example, n.definitionSource, n.dataProviders
"""
graph.run(query).to_data_frame()

,n.name,n.shortDescription,n.description,n.example,n.definitionSource,n.dataProviders
0,Location,Geographic location,A geographic location,"World, ..., Country, State, Country, City, Cru...",None,"[GeoNames, UNSD, USCensus, HUD, JHU]"
1,World,The World,Top level location,None,None,None
2,UNRegion,Continental regions,Continental regions according to the M49 stan...,Americas,https://unstats.un.org/unsd/methodology/m49/,[UNSD]
3,UNSubRegion,Subcontinental regions,Subcontinental regions according to the M49 st...,Latin America and the Caribbean,https://unstats.un.org/unsd/methodology/m49/,[UNSD]
4,UNIntermediateRegion,Subdivisions of subcontinental regions,Subdivisions of subcontinental regions accordi...,Caribbean,https://unstats.un.org/unsd/methodology/m49/,[UNSD]
5,Country,Countries and dependent Territories,Countries and dependent Territories defined b...,United States,http://www.geonames.org/,[GeoNames]
6,Admin1,"State, Province, Municipality","First administrative divisions, e.g, State, Pr...",California,http://www.geonames.org/,"[GeoNames, USCensus]"
7,Admin2,County,Second administrative divisions: County in the US,San Diego County,http://www.geonames.org/,"[GeoNames, USCensus]"
8,City,City,City,San Diego,http://www.geonames.org/,"[GeoNames, USCensus]"
9,PostalCode,Postal Code,"E.g., a ZIP Code is a postal code used by the ...",92121,http://purl.obolibrary.org/obo/OPMI_0000120,"[GeoNames, HUD]"


### Run a fuzzy full text search for bioentities
Results are ordered by match score.

In [5]:
query = """
CALL db.index.fulltext.queryNodes("bioentities", "Spike") YIELD node, score
RETURN node.name, labels(node), score
"""
df = graph.run(query).to_data_frame()
df.head(25)

,node.name,labels(node),score
0,Spike glycoprotein,[Protein],5.119284
1,Spike glycoprotein,[Protein],5.119284
2,Spike glycoprotein,[ProteinName],5.119284
3,Spike glycoprotein,[ProteinName],5.119284
4,Spike protein S1,[Protein],4.618233
5,Spike protein S2,[Protein],4.618233
6,Spike protein S2',[Protein],4.618233
7,Spike protein S1,[ProteinName],4.618233
8,Spike protein S2,[ProteinName],4.618233
9,Spike protein S2',[ProteinName],4.618233


### Run an exact full text search for bioentities
For exact matches, enclose the string with `\"`.

In [6]:
query = """
CALL db.index.fulltext.queryNodes("bioentities", '\"Spike glycoprotein\"') YIELD node
RETURN node.name, labels(node), node.accession, node.full
"""
df = graph.run(query).to_data_frame()
df.head(25)

,node.name,labels(node),node.accession,node.full
0,Spike glycoprotein,[Protein],uniprot:P0DTC2,None
1,Spike glycoprotein,[Protein],uniprot:P0DTC2,None
2,Spike glycoprotein,[ProteinName],uniprot:P0DTC2,None
3,Spike glycoprotein,[ProteinName],uniprot:P0DTC2,None


### List Organisms in KG

In [7]:
query = """
MATCH (p:Organism)
RETURN p.name as name, p.scientificName as scientificName, p.id as taxonomy
"""
graph.run(query).to_data_frame()

,name,scientificName,taxonomy
0,SARS-CoV-2,Severe acute respiratory syndrome coronavirus 2,taxonomy:2697049
1,MERS-CoV,Middle East respiratory syndrome-related coron...,taxonomy:1335626
2,SARS-CoV,Severe acute respiratory syndrome-related coro...,taxonomy:694009
3,human,Homo sapiens,taxonomy:9606
4,house mouse,Mus musculus,taxonomy:10090
5,intermediate horseshoe bat,Rhinolophus affinis,taxonomy:59477
6,Malayan horseshoe bat,Rhinolophus malayanus,taxonomy:608659
7,horseshoe bat,Rhinolophus,taxonomy:49442
8,Malayan pangolin,Manis javanica,taxonomy:9974
9,palm civet,Paradoxurus,taxonomy:71116


### List Coronavirus Outbreaks

In [8]:
query = """
MATCH (p:Pathogen)-[:CAUSES]->(o:Outbreak)
RETURN p.name as name, p.scientificName as scientificName, p.id as taxonomy, o.id as outbreak, o.startDate as startDate
"""
graph.run(query).to_data_frame()

,name,scientificName,taxonomy,outbreak,startDate
0,SARS-CoV-2,Severe acute respiratory syndrome coronavirus 2,taxonomy:2697049,COVID-19,2019
1,MERS-CoV,Middle East respiratory syndrome-related coron...,taxonomy:1335626,MERS,2012
2,SARS-CoV,Severe acute respiratory syndrome-related coro...,taxonomy:694009,SARS,2003


### List Strains that are mentioned in PubMed Central Articles

In [9]:
query = """
MATCH (p:Publication)-[:MENTIONS]->(s:Strain)<-[:CARRIES]-(h:Host)
RETURN p.id as pmc, s.name as name, s.collectionDate  as collectionDate, h.name as host, h.id as host_id
ORDER by s.collectionDate
"""
graph.run(query).to_data_frame().head(20)

,pmc,name,collectionDate,host,host_id
0,pmc:PMC7166309,BetaCoV/pangolin/Guangxi/P5E/2017,2017-01-01,Malayan pangolin,taxonomy:9974
1,pmc:PMC7166309,BetaCoV/pangolin/Guangxi/P4L/2017,2017-01-01,Malayan pangolin,taxonomy:9974
2,pmc:PMC7166309,BetaCoV/pangolin/Guangxi/P5L/2017,2017-01-01,Malayan pangolin,taxonomy:9974
3,pmc:PMC7166309,BetaCoV/pangolin/Guangxi/P1E/2017,2017-01-01,Malayan pangolin,taxonomy:9974
4,pmc:PMC7166309,BetaCoV/pangolin/Guangxi/P2V/2017,2017-01-01,Malayan pangolin,taxonomy:9974
5,pmc:PMC7228214,BetaCoV/pangolin/Guangxi/P4L/2017,2017-01-01,Malayan pangolin,taxonomy:9974
6,pmc:PMC7211627,BetaCoV/pangolin/Guangxi/P5L/2017,2017-01-01,Malayan pangolin,taxonomy:9974
7,pmc:PMC7166309,BetaCoV/pangolin/Guandong/1/2020,2019-01-01,Malayan pangolin,taxonomy:9974
8,pmc:PMC7228214,BetaCoV/pangolin/Guandong/1/2020,2019-01-01,Malayan pangolin,taxonomy:9974
9,pmc:PMC7205519,BetaCoV/pangolin/Guandong/1/2020,2019-01-01,Malayan pangolin,taxonomy:9974


### List Gene and Protein information for Reference Genome
This query lists the genes and proteins encoded by the SARS-CoV-2 reference genome. This is the first genome of SARS-CoV-2 collected in Wuhan on Dec. 5, 2019.

In [10]:
query = """
MATCH (s:Strain)-[:HAS]->(g:Gene)-[:ENCODES]->(p:Protein)
RETURN s.id as referenceGenome, s.name as name, s.collectionDate  as collectionDate, 
       g.name as gene, g.id as geneId, p.name as protein, p.id as protein_id 
ORDER by s.collectionDate
"""
graph.run(query).to_data_frame()

,referenceGenome,name,collectionDate,gene,geneId,protein,protein_id
0,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-20,ORF1ab,ncbiprotein:NC_045512-266-21555,Replicase polyprotein 1ab,md5:e6608b50fcd6e004708a875615ddf2d9
1,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-20,ORF1ab,ncbiprotein:NC_045512-266-13483,Replicase polyprotein 1a,md5:e781b58591b8dbdd15f84dcbdec82105
2,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-20,S,ncbiprotein:NC_045512-21563-25384,Spike glycoprotein,md5:4c35f09aac2f7be4f3cffd30c6aecac8
3,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-20,ORF3a,ncbiprotein:NC_045512-25393-26220,Protein 3a,md5:f5c8b89ceac3f14e456577557df1ef40
4,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-20,E,ncbiprotein:NC_045512-26245-26472,Envelope small membrane protein,md5:375e0f905c315e06a99c80b736c125d2
5,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-20,M,ncbiprotein:NC_045512-26523-27191,Membrane protein,md5:1cd6abff79ad3633e17582eb0e576539
6,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-20,ORF6,ncbiprotein:NC_045512-27202-27387,Non-structural protein 6,md5:90b50e0be9abd893bd32b163d6933f0c
7,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-20,ORF7a,ncbiprotein:NC_045512-27394-27759,Protein 7a,md5:f65213344e2e68de1cae4feb9c5e07b1
8,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-20,ORF7b,ncbiprotein:NC_045512-27756-27887,Protein non-structural 7b,md5:c7f0179da4ca26456ee905081f485cc8
9,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-20,ORF8,ncbiprotein:NC_045512-27894-28259,Non-structural protein 8,md5:3a77ad9207d29beacfbf6f18b0da9e26
